In [44]:
import string
import re
from pickle import dump
from unicodedata import normalize
from numpy import array

# load doc into memory
def load_doc(filename):
    # open the file as read only
    file = open(filename, mode='rt', encoding='utf-8')
    # read all text
    text = file.read()
    # close the file
    file.close()
    return text

# split a loaded document into sentences
def to_pairs(doc):
    lines = doc.strip().split('\n')
    pairs = [line.split('\t') for line in  lines]
    return pairs

# clean a list of lines
def clean_pairs(lines):
    cleaned = list()
    # prepare regex for char filtering
    re_print = re.compile('[^%s]' % re.escape(string.printable))
    # prepare translation table for removing punctuation
    table = str.maketrans('', '', string.punctuation)
    for pair in lines:
        clean_pair = list()
        for line in pair:
            # normalize unicode characters
            line = normalize('NFD', line).encode('ascii', 'ignore')
            line = line.decode('UTF-8')
            # tokenize on white space
            line = line.split()
            # convert to lowercase
            line = [word.lower() for word in line]
            # remove punctuation from each token
#             line = [word.translate(table) for word in line]
            # remove non-printable chars form each token
#             line = [re_print.sub('', w) for w in line]
            # remove tokens with numbers in them
#             line = [word for word in line if word.isalpha()]
            # store as string
            clean_pair.append(' '.join(line))
        cleaned.append(clean_pair)
    return array(cleaned)

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

# load dataset
filename = 'translate.txt'
doc = load_doc(filename)
# # split into english-german pairs
pairs = to_pairs(doc)
# clean sentences
clean_pairs = clean_pairs(pairs)
print(clean_pairs.shape)
# save clean pairs to file
save_clean_data(clean_pairs, 'english-abap.pkl')
# spot check
for i in range(34):
    print('[%s] => [%s]' % (pairs[i][0], pairs[i][1]))

(34, 2)
Saved: english-abap.pkl
[Select all from employee] => [Get all the employee details from employee table]
[Select all from EmployeeLeaves] => [Get all the leave details from EmployeeLeaves]
[Select all from EKKO] => [Get All the Purchase orders number from Table EKKO]
[Select all from PA0001] => [Get all details from PA0001 Table]
[Select all from VBAP] => [Get all the Sales order data from table VBAP]
[Select all from LIKP] => [Get all deliveries from table LIKP]
[Select all from VBRK] => [Get all invoices from table VBRK]
[Select all from MARA] => [Get all material details from table MARA]
[Select all from EKPO] => [Get all Purchasing Document Item from EKPO table]
[Select all from MARC] => [Get all Plant Data for Material from MARC Table]
[Select all from KNA1] => [Get all General Data in Customer Master from KNA1 Table]
[select all from MAKT] => [Get all Material Descriptions from table MAKT]
[select empid from employee] => [get empid from employee table]
[select doj from em

In [45]:
from pickle import load
from pickle import dump
from numpy.random import rand
from numpy.random import shuffle

# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

# save a list of clean sentences to file
def save_clean_data(sentences, filename):
    dump(sentences, open(filename, 'wb'))
    print('Saved: %s' % filename)

# load dataset
raw_dataset = load_clean_sentences('english-abap.pkl')
print(raw_dataset.shape)
# reduce dataset size
n_sentences = 34
dataset = raw_dataset[:n_sentences, :]
# print(dataset)
# random shuffle
shuffle(raw_dataset)
# split into train/test
train, test = dataset[:25], dataset[25:]
# save
save_clean_data(raw_dataset, 'english-abap-both.pkl')
save_clean_data(train, 'english-abap-train.pkl')
save_clean_data(test, 'english-abap-test.pkl')

(34, 2)
Saved: english-abap-both.pkl
Saved: english-abap-train.pkl
Saved: english-abap-test.pkl


In [46]:
from pickle import load
from numpy import array
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.utils import to_categorical
from keras.utils.vis_utils import plot_model
from keras.models import Sequential
from keras.layers import LSTM
from keras.layers import Dense
from keras.layers import Embedding
from keras.layers import RepeatVector
from keras.layers import TimeDistributed
from keras.callbacks import ModelCheckpoint
%matplotlib inline
# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# one hot encode target sequence
def encode_output(sequences, vocab_size):
    ylist = list()
    for sequence in sequences:
        encoded = to_categorical(sequence, num_classes=vocab_size)
        ylist.append(encoded)
    y = array(ylist)
    y = y.reshape(sequences.shape[0], sequences.shape[1], vocab_size)
    return y

# define NMT model
def define_model(src_vocab, tar_vocab, src_timesteps, tar_timesteps, n_units):
    model = Sequential()
    model.add(Embedding(src_vocab, n_units, input_length=src_timesteps, mask_zero=True))
    model.add(LSTM(n_units))
    model.add(RepeatVector(tar_timesteps))
    model.add(LSTM(n_units, return_sequences=True))
    model.add(TimeDistributed(Dense(tar_vocab, activation='softmax')))
    return model

# load datasets
dataset = load_clean_sentences('english-abap-both.pkl')
train = load_clean_sentences('english-abap-train.pkl')
test = load_clean_sentences('english-abap-test.pkl')
# print(dataset)
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
print('ABAP Vocabulary Size: %d' % eng_vocab_size)
print('ABAP Max Length: %d' % (eng_length))
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print('Englis Vocabulary Size: %d' % ger_vocab_size)
print('English Max Length: %d' % (ger_length))
# print(dataset[:, 0])
# print(dataset[:, 1])
# prepare training data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
trainY = encode_sequences(eng_tokenizer, eng_length, train[:, 0])
trainY = encode_output(trainY, eng_vocab_size)
# prepare validation data
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])
testY = encode_sequences(eng_tokenizer, eng_length, test[:, 0])
testY = encode_output(testY, eng_vocab_size)

# define model
model = define_model(ger_vocab_size, eng_vocab_size, ger_length, eng_length, 256)
model.compile(optimizer='adam', loss='categorical_crossentropy')
# summarize defined model
print(model.summary())
plot_model(model, to_file='model.png', show_shapes=True)
# fit model
filename = 'model.h5'
checkpoint = ModelCheckpoint(filename, monitor='val_loss', verbose=1, save_best_only=True, mode='min')
model.fit(trainX, trainY, epochs=20, batch_size=3, validation_data=(testX, testY), callbacks=[checkpoint], verbose=2)

ABAP Vocabulary Size: 32
ABAP Max Length: 4
Englis Vocabulary Size: 57
English Max Length: 10
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_17 (Embedding)     (None, 10, 256)           14592     
_________________________________________________________________
lstm_33 (LSTM)               (None, 256)               525312    
_________________________________________________________________
repeat_vector_17 (RepeatVect (None, 4, 256)            0         
_________________________________________________________________
lstm_34 (LSTM)               (None, 4, 256)            525312    
_________________________________________________________________
time_distributed_17 (TimeDis (None, 4, 32)             8224      
Total params: 1,073,440
Trainable params: 1,073,440
Non-trainable params: 0
_________________________________________________________________
None
Train on 25 samples, validate on 

In [47]:
from pickle import load
from numpy import array
from numpy import argmax
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import load_model
from nltk.translate.bleu_score import corpus_bleu

# load a clean dataset
def load_clean_sentences(filename):
    return load(open(filename, 'rb'))

# fit a tokenizer
def create_tokenizer(lines):
    tokenizer = Tokenizer()
    tokenizer.fit_on_texts(lines)
    return tokenizer

# max sentence length
def max_length(lines):
    return max(len(line.split()) for line in lines)

# encode and pad sequences
def encode_sequences(tokenizer, length, lines):
    # integer encode sequences
    X = tokenizer.texts_to_sequences(lines)
    # pad sequences with 0 values
    X = pad_sequences(X, maxlen=length, padding='post')
    return X

# map an integer to a word
def word_for_id(integer, tokenizer):
    for word, index in tokenizer.word_index.items():
        if index == integer:
            return word
    return None

# generate target given source sequence
def predict_sequence(model, tokenizer, source):
    prediction = model.predict(source, verbose=0)[0]
    integers = [argmax(vector) for vector in prediction]
    target = list()
    for i in integers:
        word = word_for_id(i, tokenizer)
        if word is None:
            break
        target.append(word)
    return ' '.join(target)

# evaluate the skill of the model
def evaluate_model(model, tokenizer, sources, raw_dataset):
    actual, predicted = list(), list()
    for i, source in enumerate(sources):
        # translate encoded source text
        source = source.reshape((1, source.shape[0]))
        translation = predict_sequence(model, eng_tokenizer, source)
        raw_target, raw_src = raw_dataset[i]
        if i < 10:
            print('src=[%s], target=[%s], predicted=[%s]' % (raw_src, raw_target, translation))
        actual.append(raw_target.split())
        predicted.append(translation.split())
#     # calculate BLEU score
#     print('BLEU-1: %f' % corpus_bleu(actual, predicted, weights=(1.0, 0, 0, 0)))
#     print('BLEU-2: %f' % corpus_bleu(actual, predicted, weights=(0.5, 0.5, 0, 0)))
#     print('BLEU-3: %f' % corpus_bleu(actual, predicted, weights=(0.3, 0.3, 0.3, 0)))
#     print('BLEU-4: %f' % corpus_bleu(actual, predicted, weights=(0.25, 0.25, 0.25, 0.25)))

# load datasets
dataset = load_clean_sentences('english-abap-both.pkl')
train = load_clean_sentences('english-abap-train.pkl')
test = load_clean_sentences('english-abap-test.pkl')
# prepare english tokenizer
eng_tokenizer = create_tokenizer(dataset[:, 0])
eng_vocab_size = len(eng_tokenizer.word_index) + 1
eng_length = max_length(dataset[:, 0])
# prepare german tokenizer
ger_tokenizer = create_tokenizer(dataset[:, 1])
ger_vocab_size = len(ger_tokenizer.word_index) + 1
ger_length = max_length(dataset[:, 1])
print(ger_length)
# prepare data
trainX = encode_sequences(ger_tokenizer, ger_length, train[:, 1])
testX = encode_sequences(ger_tokenizer, ger_length, test[:, 1])

# load model
model = load_model('model.h5')
# test on some training sequences
print('train')
evaluate_model(model, eng_tokenizer, trainX, train)
# test on some test sequences
print('test')
evaluate_model(model, eng_tokenizer, testX, test)

10
train
src=[get publicholidays from employeeleaves table], target=[select publicholidays from employeeleaves], predicted=[select all from mara]
src=[get empid from employee table], target=[select empid from employee], predicted=[select from from kna1]
src=[return pno from pa0001], target=[select pno from pa0001], predicted=[select from from kna1]
src=[get matdesc from makt], target=[select matdesc from makt], predicted=[select from from makt]
src=[select quantity from makt table], target=[select quantity from makt], predicted=[select all from mara]
src=[get all the sales order data from table vbap], target=[select all from vbap], predicted=[select all from marc]
src=[get vbakno from table vbrk], target=[select vbakno from vbrk], predicted=[select all from mara]
src=[get lkapno table likp], target=[select lkapno from likp], predicted=[select from from ekko]
src=[fetch invoiceno from vbrk], target=[select invoiceno from vbrk], predicted=[select all from vbrk]
src=[get all plant data fo